# Metadata

**Customer Domain:** - Chemical Manufacturing

**Customer Subdomain:** - Chemical Manufacturing

**Domain:** - Complex Instruction Following

**Taxonomy Type:** - Tabular Data

**L2 Taxonomy Type:** -  Data Transformation (Normalization/Standardization)

**Conversation Length:** - 3 Turn Tasks

**Scenario:** - [Company], a specialty chemical manufacturer, needs to optimize the mixing process for a new polymer blend, [PolymerBlendX]. They've been collecting batch data from various sensors across their reactor. However, this data is currently stored in a fragmented and inconsistent manner, hindering effective analysis. Specifically, the input data comes from two sources: Source A, a legacy system, reports temperature readings in Celsius with varying levels of precision (e.g., "25.1", "100", "78.675"), and pressure in PSI rounded to the nearest integer. Source B, a newer IoT sensor network, provides temperature in Fahrenheit with two decimal places of precision and pressure in bar, also with two decimal places. These readings are recorded as strings within comma separated value files, with the first column indicating the timestamp in 'YYYY-MM-DD HH:MM:SS' format and inconsistent column order across files.

To effectively analyze the batch data and optimize [PolymerBlendX]'s mixing process, [Company] requires a data transformation pipeline. This pipeline must accept the raw CSV strings from both Source A and Source B. The data needs to be normalized, converting all temperature readings to a common unit (Kelvin) and pressure readings to a common unit (Pascal). Furthermore, the varying precision needs to be standardized to two decimal places for both temperature and pressure, and the timestamp format consolidated. Finally, the transformed data, including timestamp, standardized temperature, and standardized pressure, should be presented in a single, unified tabular format suitable for further analysis by data scientists.




**[system]**

---

You are an AI assistant helping a chemical manufacturer prepare and normalize fragmented batch sensor data from multiple sources. The goal is to support optimizing the mixing process for their new polymer blend. The user will provide one or more datasets directly into the conversation. You are responsible for validating the input, clarifying unit formats, detecting anomalies, standardizing the data, and supporting optional data segmentation or cleanup.

When the user initiates a request, begin by asking them to share the CSV-formatted data. Each dataset must include at least three columns: a timestamp, a temperature value, and a pressure value. If any column is missing, ask the user to provide the required columns. The timestamp column may appear in different formats, so after receiving the data, ask the user to describe the format used. For example, a user might say the timestamp format is "YYYY-MM-DD HH:MM:SS" or "DD/MM/YYYY HH:MM". Do not assume the format; confirm with the user to ensure accurate parsing.

The temperature and pressure columns may or may not specify the units in their headers. If the units are mentioned, such as "Temperature_C" or "Pressure_bar", extract them directly. If not, ask the user to clarify which units were used for each relevant column. Only temperature values in Celsius or Fahrenheit are accepted. If the user provides any other temperature unit, inform them that it is not supported, and you cannot proceed until a supported unit is confirmed. Only pressure values in PSI or bar are accepted. If any other pressure unit is used, inform the user that you cannot continue until a valid unit is provided.

After getting the datasets and necessary inputs, scan for rows that contain missing or invalid values. If any non-numeric data appears in the temperature or pressure columns, flag those rows. If any timestamp values cannot be parsed based on the provided format, inform the user and show the affected entries. Also, detect and report duplicate timestamps so that the user can decide whether to keep or remove them.

Once the data passes validation, proceed with standardization. Convert all temperature values to Kelvin. If the original unit is Celsius, use the formula:
$$  
K = C + 273.15  
$$
If the original unit is Fahrenheit, use the formula:
$$  
K = \left(\frac{F - 32}{1.8}\right) + 273.15  
$$
For pressure, convert all values to Pascal. If the original unit is PSI, use  
$$
Pa = PSI \times 6894.76  
$$
If the unit is bar, use  
$$
Pa = bar \times 100000  
$$
After conversion, round all temperature and pressure values to two decimal places. Reformat all timestamps to the unified format "YYYY-MM-DD HH:MM:SS".

After standardization, perform anomaly detection. Use the following thresholds for detecting outliers: temperature values in Kelvin must be between 250 and 700, and pressure values in Pascal must fall between 50000 and 2000000. Any row that contains a value outside these ranges should be flagged as a suspected anomaly. Present these rows to the user along with their row number, timestamp, and offending values, and ask the user whether to keep or remove them from the final dataset.

Present the calculation and standardization in the following format, starting with an intro line giving an overview of the processing:
```markdown
[For each dataset]
### Dataset [Dataset Number] (Temperature in [unit], Pressure in [unit])  
Formulas used:  
[Present the formula]

| timestamp   | temperature ([unit]) | pressure ([unit]) | temperature_K calculation   | pressure_Pa calculation   |
|-------------|----------------------|-------------------|-----------------------------|---------------------------|
| [timestamp] | [temperature]        | [pressure]        | [temperature_K calculation] | [pressure_Pa calculation] |

### Combined Dataset (Temperature in Kelvin, Pressure in Pascal, sorted by timestamp)  

| timestamp   | temperature_K   | pressure_Pa   |  
|-------------|-----------------|---------------|  
| [timestamp] | [temperature_K] | [pressure_Pa] |
  
### Anomaly Check  
  
Thresholds:  
- Temperature in Kelvin must be between 250 and 700  
- Pressure in Pascal must be between 50000 and 2000000  
  
[Check for the anomaly based on the threshold and inform the user if the dataset is ready for further processing.]
```

After presenting the combined dataset, ask follow-up questions if the user wants an optional enhancement. You can segment the dataset based on specific timestamp ranges, such as daily intervals, work shifts, or custom date ranges provided by the user. If the batch_id column is present, you can also segment the data by batch. Additionally, offer to handle missing values, if any, by applying imputation strategies, such as using the column mean, median, or forward-fill logic, depending on what the user prefers.

**[user]**  
  
We’ve been collecting data from multiple reactors for PolymerBlendX, but the format is inconsistent across sources. I have a few CSV datasets with timestamp, temperature, and pressure readings. Some temperature values are in Fahrenheit, some in Celsius, and pressures vary between PSI and bar. Can you help me clean and standardize this data?

**[assistant]**  
  
Of course, I’m here to help. Please share the raw CSV-formatted data. It should include at least three columns: timestamp, temperature, and pressure. I’ll take a look as soon as you send it.

**[user]**  
  
Here are two datasets from different sources:  
  
timestamp,temp,pressure  
2025-04-01 08:00:00,78.8,14.7  
2025-04-01 08:10:00,80.2,14.6  
2025-04-01 08:20:00,81.0,14.5  
2025-04-01 08:30:00,82.1,14.5  
2025-04-01 08:40:00,83.0,14.4  
2025-04-01 08:50:00,83.5,14.3  
2025-04-01 09:00:00,84.1,14.2  
2025-04-01 09:10:00,84.8,14.1  
2025-04-01 09:20:00,85.3,14.0  
2025-04-01 09:30:00,85.7,13.9  
  
And the second one is:  
timestamp,temp,pressure  
2025-04-01 08:05:00,25.1,101  
2025-04-01 08:15:00,25.4,100  
2025-04-01 08:25:00,25.7,99  
2025-04-01 08:35:00,26.0,98  
2025-04-01 08:45:00,26.2,98  
2025-04-01 08:55:00,26.6,97  
2025-04-01 09:05:00,26.9,96  
2025-04-01 09:15:00,27.1,95  
2025-04-01 09:25:00,27.4,95  
2025-04-01 09:35:00,27.6,94

**[assistant]**  
  
To proceed, I need to confirm the unit used for the temperature column in each dataset. Is it Celsius or Fahrenheit? Also, what unit is used for the pressure column? Is it PSI or bar?  
  
Please also describe the format of the timestamp values. For example, is it written as "YYYY-MM-DD HH:MM:SS" or something else? This will help ensure accurate parsing.

**[user]**  
  
The first dataset has temperature in Fahrenheit and pressure in bar.  
The second dataset has temperature in Celsius and pressure in PSI.  
And yes, you guessed the timestamp format correctly.

**[assistant]**  
  
The datasets have been processed and standardized. Temperature values have been converted to Kelvin and pressure values to Pascal. Each transformation is shown in detail below, along with the final combined dataset sorted by timestamp and a validation check against defined thresholds.  
  
### Dataset 1 (Temperature in Fahrenheit, Pressure in bar)  
Formulas used:  
$$\text{temperature_K} = \left(\frac{F - 32}{1.8}\right) + 273.15$$  
$$\text{pressure_Pa} = \text{bar} \times 100000$$  
  
| timestamp | temperature (F) | pressure (bar) | temperature_K calculation | pressure_Pa calculation |  
|---------------------|----------|----------------|-------------------------------------------------------------|-------------------------------|  
| 2025-04-01 08:00:00 | 78.8     | 14.7           | $((78.8 - 32) / 1.8) + 273.15 = 299.15$      | $14.7 \times 100000 = 1470000.00$ |
| 2025-04-01 08:10:00 | 80.2     | 14.6           | $((80.2 - 32) / 1.8) + 273.15 = 299.93$      | $14.6 \times 100000 = 1460000.00$ |
| 2025-04-01 08:20:00 | 81.0     | 14.5           | $((81.0 - 32) / 1.8) + 273.15 = 300.37$      | $14.5 \times 100000 = 1450000.00$ |
| 2025-04-01 08:30:00 | 82.1     | 14.5           | $((82.1 - 32) / 1.8) + 273.15 = 300.98$      | $14.5 \times 100000 = 1450000.00$ |
| 2025-04-01 08:40:00 | 83.0     | 14.4           | $((83.0 - 32) / 1.8) + 273.15 = 301.48$      | $14.4 \times 100000 = 1440000.00$ |
| 2025-04-01 08:50:00 | 83.5     | 14.3           | $((83.5 - 32) / 1.8) + 273.15 = 301.76$      | $14.3 \times 100000 = 1430000.00$ |
| 2025-04-01 09:00:00 | 84.1     | 14.2           | $((84.1 - 32) / 1.8) + 273.15 = 302.09$      | $14.2 \times 100000 = 1420000.00$ |
| 2025-04-01 09:10:00 | 84.8     | 14.1           | $((84.8 - 32) / 1.8) + 273.15 = 302.48$      | $14.1 \times 100000 = 1410000.00$ |
| 2025-04-01 09:20:00 | 85.3     | 14.0           | $((85.3 - 32) / 1.8) + 273.15 = 302.76$      | $14.0 \times 100000 = 1400000.00$ |
| 2025-04-01 09:30:00 | 85.7     | 13.9           | $((85.7 - 32) / 1.8) + 273.15 = 302.98$      | $13.9 \times 100000 = 1390000.00$ |
  
### Dataset 2 (Temperature in Celsius, Pressure in PSI)  
Formulas used:  
$$\text{temperature_K} = C + 273.15$$  
$$\text{pressure_Pa} = \text{PSI} \times 6894.76$$  
  
| timestamp | temperature (C) | pressure (PSI) | temperature_K calculation | pressure_Pa calculation |  
|---------------------|----------|----------------|--------------------------------------|----------------------------------------|  
| 2025-04-01 08:05:00 | 25.1     | 101            | $25.1 + 273.15 = 298.25$         | $101 \times 6894.76 = 696370.76$ |
| 2025-04-01 08:15:00 | 25.4     | 100            | $25.4 + 273.15 = 298.55$         | $100 \times 6894.76 = 689476.00$ |
| 2025-04-01 08:25:00 | 25.7     | 99             | $25.7 + 273.15 = 298.85$         | $99 \times 6894.76 = 682581.24$ |
| 2025-04-01 08:35:00 | 26.0     | 98             | $26.0 + 273.15 = 299.15$         | $98 \times 6894.76 = 675686.48$ |
| 2025-04-01 08:45:00 | 26.2     | 98             | $26.2 + 273.15 = 299.35$         | $98 \times 6894.76 = 675686.48$ |
| 2025-04-01 08:55:00 | 26.6     | 97             | $26.6 + 273.15 = 299.75$         | $97 \times 6894.76 = 668791.72$ |
| 2025-04-01 09:05:00 | 26.9     | 96             | $26.9 + 273.15 = 300.05$         | $96 \times 6894.76 = 661896.96$ |
| 2025-04-01 09:15:00 | 27.1     | 95             | $27.1 + 273.15 = 300.25$         | $95 \times 6894.76 = 655002.20$ |
| 2025-04-01 09:25:00 | 27.4     | 95             | $27.4 + 273.15 = 300.55$         | $95 \times 6894.76 = 655002.20$ |
| 2025-04-01 09:35:00 | 27.6     | 94             | $27.6 + 273.15 = 300.75$         | $94 \times 6894.76 = 648107.44$ |
  
### Combined Dataset (Temperature in Kelvin, Pressure in Pascal, sorted by timestamp)  
  
| timestamp | temperature_K | pressure_Pa |  
|---------------------|----------------|--------------|  
| 2025-04-01 08:00:00 | 299.15         | 1470000.00   |
| 2025-04-01 08:05:00 | 298.25         | 696370.76    |
| 2025-04-01 08:10:00 | 299.93         | 1460000.00   |
| 2025-04-01 08:15:00 | 298.55         | 689476.00    |
| 2025-04-01 08:20:00 | 300.37         | 1450000.00   |
| 2025-04-01 08:25:00 | 298.85         | 682581.24    |
| 2025-04-01 08:30:00 | 300.98         | 1450000.00   |
| 2025-04-01 08:35:00 | 299.15         | 675686.48    |
| 2025-04-01 08:40:00 | 301.48         | 1440000.00   |
| 2025-04-01 08:45:00 | 299.35         | 675686.48    |
| 2025-04-01 08:50:00 | 301.76         | 1430000.00   |
| 2025-04-01 08:55:00 | 299.75         | 668791.72    |
| 2025-04-01 09:00:00 | 302.09         | 1420000.00   |
| 2025-04-01 09:05:00 | 300.05         | 661896.96    |
| 2025-04-01 09:10:00 | 302.48         | 1410000.00   |
| 2025-04-01 09:15:00 | 300.25         | 655002.20    |
| 2025-04-01 09:20:00 | 302.76         | 1400000.00   |
| 2025-04-01 09:25:00 | 300.55         | 655002.20    |
| 2025-04-01 09:30:00 | 302.98         | 1390000.00   |
| 2025-04-01 09:35:00 | 300.75         | 648107.44    |
  
  
### Anomaly Check  
  
Thresholds:  
- Temperature in Kelvin must be between 250 and 700  
- Pressure in Pascal must be between 50000 and 2000000  
  
All values in the combined dataset fall within these defined thresholds. No anomalies were detected. The data is clean and ready for further processing.


Would you like to segment the dataset by specific time ranges such as day, shift, or a custom interval?